<a href="https://colab.research.google.com/github/rhayakawa/predict-admm-cs/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# predict-admm-cs

This is a python demo code for the following paper:  
R. Hayakawa, "Asymptotic performance prediction for ADMM-based compressed sensing," IEEE Transactions on Signal Processing, 2022.  
([IEEE Xplore](https://ieeexplore.ieee.org/document/9932009), [arXiv](https://arxiv.org/abs/2009.08545), [GitHub](https://github.com/rhayakawa/predict-admm-cs))

## git clone

In [ ]:
!git clone https://github.com/rhayakawa/predict-admm-cs.git
%cd predict-admm-cs/code

## import modules

In [ ]:
import numpy as np
from tqdm import tqdm
from my_module import cs, cgmt, my_plot

## problem settings

We consider the reconstruction of an unknown vector $\mathbf{x} \in \mathbb{R}^{N}$ from its linear measurements 
\begin{align*}
    \mathbf{y} = \mathbf{A} \mathbf{x} + \mathbf{v} \in \mathbb{R}^{M}, 
\end{align*}
where $\mathbf{A} \in \mathbb{R}^{M \times N}$ is a Gaussian measurement matrix and $\mathbf{v} \in \mathbb{R}^{M}$ is an additive Gaussian noise vector. 

### parameter of the reconstruction problem
- N: dimension of the unknown vector $\mathbf{x}$
- delta: measurement ratio $M/N$
- distribution: probability distribution of the unknown vector
    - 'sparse': Bernoulli-Gaussian distirbution $p_{0} \delta_{0}(x) + (1 - p_{0}) p_{\mathrm{Gaussian}}(x)$
    - 'binary': binary distribution with $1$ and $-1$
- p0: probability of zero (i.e., $p_{0}$) for sparse unknown vectors
- sigma2_v: noise variance $\sigma_{\text{v}}^{2}$ of the elements of $\mathbf{v}$

### other parameter

- measure: performance measure
    - 'MSE': mean squared error (for sparse vector)
    - 'SER': symbol error rate (for binary vector)


In [ ]:
prob_param = cs.ProbParam(N=500,  # dimension of the unknown vector
                          delta=0.9,  # measurement ratio
                          distribution='sparse',  # distribution of the unknown vector
                          p0=0.8,  # sparsity of the unknown vector
                          sigma2_v=1e-3)  # noise variance
measure = 'MSE'  # performance measure

## asymptotic performance obtained by CGMT (Convex Gaussian Min-max Theorem) framework

We consider the following regularized optimization problem:
\begin{align*}
    \hat{\mathbf{x}} = \underset{\mathbf{s} \in \mathbb{R}^{N}}{\mathrm{arg\ min}} 
    \left\{ 
    \frac{1}{2} \left\| \mathbf{y} - \mathbf{A} \mathbf{s}   \right\|_{2}^{2} + \lambda f( \mathbf{s} )
    \right\}. 
\end{align*}

Using CGMT framework, we can obtain the asymptotically optimal error performance of $\hat{\mathbf{x}}$ and the corresponding regularization paramter $\lambda$. 

In [ ]:
sample_size = 100000  # sample size for the prediction (>= 100000 is better)
performance_opt, lmd_opt = cgmt.optimal_performance(prob_param, sample_size=sample_size, measure=measure)

## algorithm settings

The algorithm of ADMM (Alternating Direction Method of Multipliers) for the above optimization problem can be written as follows:
\begin{align*}
    \mathbf{s}^{(k+1)} 
    &= 
    \left( \mathbf{A}^{\top} \mathbf{A} + \rho \mathbf{I} \right)^{-1} 
    \left( \mathbf{A}^{\top} \mathbf{y} + \rho \left( \mathbf{z}^{(k)} - \mathbf{w}^{(k)}\right) \right), \\
    \mathbf{z}^{(k+1)} 
    &= 
    \mathrm{prox}_{\frac{\lambda}{\rho} f} \left( \mathbf{s}^{(k+1)} + \mathbf{w}^{(k)} \right), \\
    \mathbf{w}^{(k+1)} 
    &= 
    \mathbf{w}^{(k)} + \mathbf{s}^{(k+1)} - \mathbf{z}^{(k+1)}.
\end{align*}

### parameters of the reconstruction algorithm

- lmd: regularization parameter $\lambda$
- rho: parameter of ADMM $\rho$
- prox: proximity operator of the regularizer
    - cs.prox_L1: proximity operator of $\ell_{1}$ regularization (soft thresholding function)
    - cs.prox_box: proximity operator for box constratint
- num_iteration: number of iterations in ADMM


In [ ]:
alg_param = cs.AlgParam(lmd=lmd_opt,  # regularization parameter
                        rho=0.1,  # parameter of ADMM
                        prox=cs.prox_L1,  # proximity operator
                        num_iteration=20)  # number of iterations

## performance prediction via proposed method

In [ ]:
array_performance_prediction = cgmt.state_evolution(prob_param, alg_param, sample_size=sample_size, measure=measure, leave=True)

## empirical reconstruction performance of ADMM

In [ ]:
num_empirical = 100  # number of samples for the empirical performance evaluation (>= 100 is better)
array_performance_empirical = cs.empirical_performance(prob_param, alg_param, num_empirical=num_empirical, measure=measure, leave=True)

## compare the empirical performance and its prediction

In [ ]:
fig, ax, _, _ = my_plot.setup()
line_width = 2
marker_size = 10

ax.plot(range(alg_param.num_iteration + 1), array_performance_empirical,
        label=rf'empirical', linestyle='', marker='o', markersize=marker_size)
ax.plot(range(alg_param.num_iteration + 1), array_performance_prediction,
        label='prediction', linestyle='-', color='k', linewidth=line_width)
ax.hlines(y=performance_opt, xmin=0, xmax=alg_param.num_iteration,
          label=f'asymptotic {measure} of optimizer', linestyle='--', color='k', linewidth=line_width)

my_plot.set_ax_property(ax,
                        yscale='log',
                        xticks=range(0, alg_param.num_iteration + 1, 5),
                        xlim_left=0,
                        xlim_right=alg_param.num_iteration,
                        xlabel='number of iterations',
                        ylabel=measure)